# Urban and rural surrounding areas selection

***
> This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0)
>
> ![Creative Commons License >](https://i.creativecommons.org/l/by/4.0/88x31.png)

## Load libraries

Here we load the required libraries to execute the notebook. Note that specific code from dedicated scripts (`utils.py`, `urban_areas.py`, ...) is also loaded here.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

import os
import numpy as np
import xarray as xr
import cf_xarray as cfxr
from pathlib import Path
import glob

from urbanmask.urban_areas import (
    Urban_vicinity,
)

from urbanmask.UHI_plot_functions import (
    compute_climatology,
    plot_climatology,
    compute_time_series,
    plot_time_series
)

from urbanmask.utils import (
    kelvin2degC,
    traverseDir,
    fix_360_longitudes,
)

In [ ]:
from lib.utils import latest_version, fix_360_longitudes
from lib.interpolater import Interpolator

## Input parameters

These parameters define the behavior of the notebook by selecting the `city`, `variable`, and other items, most of which are self-explanatory. _(Provide additional explanations here for any parameters that are not self-explanatory)._

In [ ]:
# Parameters
city = 'Naples'
name = 'Naples'
lon_city = 14.2681
lat_city = 40.8518

model = 'CERRA'
variable= 't'

urban_th = 60
urban_sur_th = 10
orog_diff = 100
sftlf_th = 70
scale = 2
min_city_size = 0
lon_lim = 1
lat_lim = 1

Create a directory to save the results for this particular city and model

In [ ]:
base_output_dir = 'results_CERRA'
output_dir = f"{base_output_dir}/{city}_{model}"
os.makedirs(output_dir, exist_ok = True)

Infer doman resolution in degrees and create filename

In [ ]:
#domain_resolution = int(domain.split('-')[1])
base_filename = f'{city}-ECMWF-ERA5_r1i1p1f1_{model}'

## Define urban areas and vicinity

### Load static variables
Load static variables, such as urban fraction (sfturf), terrain elevation (orography) or land-sea fraction (sftlf).

In [ ]:
root = '/lustre/gmeteo/WORK/DATA/C3S-CDS/'

ds_orog = xr.open_dataset(f'{root}/CERRA/fx/orog_CERRA_fx.nc')
ds_sftlf = xr.open_dataset(f'{root}/CERRA/fx/lsm_CERRA_fx.nc')
ds_sftuf = xr.open_dataset(f'{root}/CERRA-Land/fx/CERRALand_tiles_fraction.nc')

In [ ]:
# Process orography
ds_orog = ds_orog.isel(valid_time=0)
ds_orog = ds_orog.rename({'longitude': 'lon', 'latitude': 'lat'})

# Process land-sea mask, converting values for land percentage
ds_sftlf = ds_sftlf.isel(valid_time=0)
ds_sftlf['sftlf'] = ds_sftlf['lsm'] * 100
ds_sftlf = ds_sftlf.drop_vars('lsm').rename({'longitude': 'lon', 'latitude': 'lat'})

# Process land-sea mask, converting values for land percentage
ds_sftuf = ds_sftuf.isel(oneD=0)
ds_sftuf['sftuf'] = ds_sftuf['frac_urban'] * 100
#ds_sftuf = ds_sftuf.drop_vars('frac_urban').rename({'x': 'lon', 'y': 'lat'})
ds_sftuf = ds_sftuf.assign_coords({'lat': ds_orog['lat'], 'lon': ds_orog['lon']})

Add parameters to the function

In [ ]:
URBAN = Urban_vicinity(
    urban_th, 
    urban_sur_th, 
    orog_diff, 
    sftlf_th,
    scale,
    min_city_size,
    lon_city,
    lat_city,
    lon_lim,
    lat_lim,
    model)

### Load climate variable

Crop area around de city

In [ ]:
root = "/lustre/gmeteo/WORK/DATA/C3S-CDS/C3S-CICA-Atlas/v2/"
domain= "CERRA"
name= "t_CERRA_mon_198501-202112_v02.nc"

In [ ]:
root_nc = root + domain + '/' + name
var = name.split('_')[0]

ds_RCM = xr.open_dataset(root_nc, engine="netcdf4", 
                    chunks={"member": 1, "time": 120})


# Crop the area for the city using the domain resolution
# Define trimming limits
lat_min = lat_city - lat_lim
lat_max = lat_city + lat_lim
lon_min = lon_city - lon_lim
lon_max = lon_city + lon_lim

# Crop the dataset
ds_RCM = ds_RCM.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))

### Interpolation of the static variable

In [ ]:
# Interpolate data
int_attr = {'interpolation_method' : 'conservative_normed', 
            'lats' : ds_RCM.lat.values,
            'lons' : ds_RCM.lon.values,
            'var_name' : 'sftuf'
}

INTER = Interpolator(int_attr)
ds_sftuf = INTER(ds_sftuf)

int_attr = {'interpolation_method' : 'conservative_normed', 
            'lats' : ds_RCM.lat.values,
            'lons' : ds_RCM.lon.values,
            'var_name' : 'orog'
}
INTER = Interpolator(int_attr)
ds_orog = INTER(ds_orog)

int_attr = {'interpolation_method' : 'conservative_normed', 
            'lats' : ds_RCM.lat.values,
            'lons' : ds_RCM.lon.values,
            'var_name' : 'sftlf'
}
INTER = Interpolator(int_attr)
ds_sftlf = INTER(ds_sftlf)

### Create urban_mask

Define masks using the parameters above

In [ ]:
sftuf_mask, sftuf_sur_mask, orog_mask, sftlf_mask = URBAN.define_masks(
    ds_sftuf, 
    ds_orog, 
    ds_sftlf
)

### Define rural vicinity areas

We use an iterative algorithm to select the urban vicinity excluding masked areas

In [ ]:
urmask = URBAN.select_urban_vicinity(
    sftuf_mask, 
    orog_mask,
    sftlf_mask,
    sftuf_sur_mask
)

We can plot the masks and the urban vs vicinity areas. The latter are are passed via the `urban_areas` parameter. They are hidden if set to `None`.

In [ ]:
fig = URBAN.plot_static_variables(ds_sftuf, ds_orog, ds_sftlf,
                                 sftuf_mask, orog_mask, sftlf_mask,
                                 urban_areas = urmask)
fig.savefig(f"{output_dir}/urmask_{base_filename}_fx.pdf", bbox_inches='tight')

Save these masks and urban fraction to NetCDF

In [ ]:
urmask.to_netcdf(f"{output_dir}/urmask_{base_filename}_fx.nc")
ds_sftuf.rename_vars({'sftuf': 'sfturf'}).to_netcdf(f"{output_dir}/sfturf_{base_filename}_fx.nc")

## plot with urban polygon

In [ ]:
root= "/lustre/gmeteo/WORK/DATA/CORDEX-FPS-URB-RCC/nextcloud/CORDEX-CORE-WG/GHS_FUA_UCD"
ucdb_info = gpd.read_file(root + '/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg')
ucdb_city = ucdb_info.query("UC_NM_MN == @city and CTR_MN_NM == 'Italy'").to_crs(crs='EPSG:4326')


In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

proj = ccrs.PlateCarree()
fig, ax = plt.subplots(subplot_kw={'projection': proj}, figsize=(12, 6))

ds_sftuf['sftuf'].plot(ax=ax)
ucdb_city.plot(ax=ax, facecolor="none", transform=proj, edgecolor="red", linewidth=2, zorder = 1000)

shapefile_path = "box/Naples_bbox.shp"
gdf = gpd.read_file(shapefile_path)
gdf.plot(ax=ax, facecolor="none", transform=proj, edgecolor="blue", linewidth=2, zorder = 1010)